In [1]:
#Install libraries
!pip install pystac-client
!pip install rio-cogeo
!pip install ipyleaflet
!pip install planetary_computer
!pip install stackstac

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.0/154.0 kB 2.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.4 MB/s eta 0:00:00


In [8]:
#Import
import ipyleaflet
import matplotlib.pyplot as plt
import numpy as np
import pystac
import planetary_computer
import requests
import rich.table
from pystac_client import Client
from rio_cogeo.cogeo import cog_translate

from IPython.display import Image

In [26]:
#get bbox
import pyproj

# Define utm coordinate system (UTM 18S)
utm = pyproj.Proj(proj='utm', zone=18, south=True, ellps='WGS84')

# define geographic coordinate system (WGS84)
wgs84 = pyproj.Proj(proj='latlong', datum='WGS84')

# Convertir las coordenadas UTM a coordenadas geográficas
lon1, lat1 = pyproj.transform(utm, wgs84, 489579.7, 8680109.8)
lon2, lat2 = pyproj.transform(utm, wgs84, 499191.2, 8694520.3)

print([lon1, lat1, lon2, lat2])


[-75.09570157701795, -11.939723561376725, -75.00742461347608, -11.80941886132244]


<ipython-input-26-c244a5239ede>:10: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon1, lat1 = pyproj.transform(utm, wgs84, 489579.7, 8680109.8)
<ipython-input-26-c244a5239ede>:11: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon2, lat2 = pyproj.transform(utm, wgs84, 499191.2, 8694520.3)


In [13]:
catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [40]:
#number of items 
search = catalog.search(
    collections=["sentinel-1-grd"],
    bbox=[-75.09570157701795, -11.939723561376725, -75.00742461347608, -11.80941886132244],
    datetime="2016-01-01T00:00:00Z/2023-12-31T23:59:59Z",
)
items = search.item_collection()
print(f"Found {len(items)} items")
item = items[0]

Found 519 items


In [45]:
items

id: S1A_IW_GRDH_1SDV_20230421T103940_20230421T104005_048193_05CB64
"bbox: [-77.22182505, -12.84253747, -74.65109212, -10.83418452]"
datetime: 2023-04-21T10:39:52.537586Z
platform: SENTINEL-1A
"s1:shape: [25651, 16833]"
end_datetime: 2023-04-21 10:40:05.036595+00:00
constellation: Sentinel-1
s1:resolution: high
s1:datatake_id: 379748
start_datetime: 2023-04-21 10:39:40.038578+00:00
s1:orbit_source: RESORB


In [41]:
#visualization
Image(url=item.assets["rendered_preview"].href)

In [34]:
#Inspect the STAC metadata

table = rich.table.Table("key", "value")
for k, v in sorted(item.properties.items()):
    table.add_row(k, str(v))

table

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ key                                   ┃ value                            ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ constellation                         │ Sentinel-1                       │
│ datetime                              │ 2016-12-31T23:25:11.196862Z      │
│ end_datetime                          │ 2016-12-31 23:25:23.696457+00:00 │
│ platform                              │ SENTINEL-1B                      │
│ s1:datatake_id                        │ 25611                            │
│ s1:instrument_configuration_ID        │ 1                                │
│ s1:orbit_source                       │ RESORB                           │
│ s1:processing_level                   │ 1                                │
│ s1:product_timeliness                 │ Fast-24h                         │
│ s1:resolution                         │ high                             │
│ s1:shape                              │ [25794, 16856]                   │
│ s1:slice_number                       │ 4                                │
│ s1:total_slices                       │ 11                               │
│ sar:center_frequency                  │ 5.405                            │
│ sar:frequency_band                    │ C                                │
│ sar:instrument_mode                   │ IW                               │
│ sar:looks_azimuth                     │ 1                                │
│ sar:looks_equivalent_number           │ 4.4                              │
│ sar:looks_range                       │ 5                                │
│ sar:observation_direction             │ right                            │
│ sar:pixel_spacing_azimuth             │ 10                               │
│ sar:pixel_spacing_range               │ 10                               │
│ sar:polarizations                     │ ['VV']                           │
│ sar:product_type                      │ GRD                              │
│ sar:resolution_azimuth                │ 22                               │
│ sar:resolution_range                  │ 20                               │
│ sat:absolute_orbit                    │ 3646                             │
│ sat:orbit_state                       │ ascending                        │
│ sat:platform_international_designator │ 2016-025A                        │
│ sat:relative_orbit                    │ 120                              │
│ start_datetime                        │ 2016-12-31 23:24:58.697267+00:00 │
└───────────────────────────────────────┴──────────────────────────────────┘

In [42]:
item.properties["sar:polarizations"]

['VV', 'VH']

In [ ]:
#Visualize the assets

from distributed import Client

client = Client(processes=False)
print(client.dashboard_link)

In [44]:
#Aqui esta el problema!!
import stackstac

ds = stackstac.stack(items, bounds_latlon=bbox, epsg=4326, resolution=100)
ds

<xarray.DataArray 'stackstac-203344811603d6736cfddcdfa9c52cec' (time: 0,
                                                                band: 0, y: 1,
                                                                x: 1)>
dask.array<fetch_raster_window, shape=(0, 0, 1, 1), dtype=float64, chunksize=(0, 0, 1, 1), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 
    id       (time) float64 
  * band     (band) float64 
  * x        (x) float64 -100.0
  * y        (y) float64 0.0
    epsg     int64 4326
Attributes:
    spec:        RasterSpec(epsg=4326, bounds=(-100, -100, 0, 0), resolutions...
    crs:         epsg:4326
    transform:   | 100.00, 0.00,-100.00|\n| 0.00,-100.00, 0.00|\n| 0.00, 0.00...
    resolution:  100

In [ ]:
vv = ds.sel(band="vh")[0].compute()